In [1]:
import dspy
from decouple import config
from typing import Literal, List

OPENROUTER_API_KEY = config("OPENROUTER_API_KEY")

In [2]:
lm = dspy.LM("openrouter/qwen/qwen-2.5-72b-instruct", api_key=OPENROUTER_API_KEY, api_base="https://openrouter.ai/api/v1",)
dspy.configure(lm=lm)

In [3]:
class Answer(dspy.Signature):
    """Answer a question based on the provided documents."""

    question: str = dspy.InputField()
    context: str = dspy.InputField()
    correct_answers: List[str] = dspy.OutputField()
    explanation: str = dspy.OutputField()

In [4]:
answer = dspy.Predict(Answer)

### Processing data into dspy examples

In [5]:
import ujson

file_path = "/home/darth/Documents/code/ramdocs/RAMDocs/src/data/split_data/RAMDocs_test_train.jsonl"

data = []
with open(file_path) as f:
    for line in f:
        line = ujson.loads(line)
        line_data = {
            'question': line['question'],
            'context': "\n".join(["Document " + str(i+1) + ": " + doc['text'] for i, doc in enumerate(line['documents'])]),
            "documents": line['documents'],
            "disambig_entity": line['disambig_entity'],
            "gold_answers": line['gold_answers'],
            "wrong_answers": line['wrong_answers']
        }
        data.append(line_data)
    # data = [ujson.loads(line) for line in f]

In [6]:
data[0]

{'question': 'What is the profession of C. Mayer?',
 'context': 'Document 1: Christa Mayer Christa Mayer is a German operatic mezzo-soprano. She is particularly known for her portrayal of Erda in Richard Wagner\'s "Ring Cycle"; a role which she has performed several times at the Bayreuth Festival and recorded on the BBC Legends Record Label. Christa Mayer was born in Sulzbach-Rosenberg, Germany in Bavaria. After graduating from the Bavarian Academy of Singing, she pursued further studies at the Munich Academy of Music where she was a pupil of tenor Thomas Moser. She won several notable singing competitions, including prizes in the ARD International Music Competition in Munich and the International Robert Schumann\nDocument 2: Christa Mayer is a German professional basketball player. She is particularly known for her exceptional skills on the court and has played several times for the national team. Christa Mayer was born in Sulzbach-Rosenberg, Germany in Bavaria. After graduating from 

In [7]:
data = [dspy.Example(**d).with_inputs('question', 'context') for d in data]

# Let's pick an `example` here from the data.
example = data[2]
example

Example({'question': 'What is the population of Sandusky Township, Ohio?', 'context': "Document 1: Sandusky Township, Crawford County, Ohio Sandusky Township is one of the sixteen townships of Crawford County, Ohio, United States. As of the 2010 census the population was 459. Located in the eastern part of the county, it borders the following townships: No municipalities are located in Sandusky Township. Sandusky Township was named from the Sandusky River, which flows through its southern part. Statewide, other Sandusky Townships are located in Richland and Sandusky counties. The township is governed by a three-member board of trustees, who are elected in November of odd-numbered years to a four-year term beginning on the following January\nDocument 2: from Wikimedia project Spanish Wikipedia located in the administrative territorial entity Crawford County 1 reference imported from Wikimedia project English Wikipedia coordinate location 40°51'5 '' N, 82°49'22 '' W 1 reference imported 

In [8]:
import random

random.Random(0).shuffle(data)
trainset, devset = data[:200], data[200:1000]

len(trainset), len(devset)

(200, 50)

### Baseline evaluation

In [9]:
pred = answer(**example.inputs())

In [10]:
example

Example({'question': 'What is the population of Sandusky Township, Ohio?', 'context': "Document 1: Sandusky Township, Crawford County, Ohio Sandusky Township is one of the sixteen townships of Crawford County, Ohio, United States. As of the 2010 census the population was 459. Located in the eastern part of the county, it borders the following townships: No municipalities are located in Sandusky Township. Sandusky Township was named from the Sandusky River, which flows through its southern part. Statewide, other Sandusky Townships are located in Richland and Sandusky counties. The township is governed by a three-member board of trustees, who are elected in November of odd-numbered years to a four-year term beginning on the following January\nDocument 2: from Wikimedia project Spanish Wikipedia located in the administrative territorial entity Crawford County 1 reference imported from Wikimedia project English Wikipedia coordinate location 40°51'5 '' N, 82°49'22 '' W 1 reference imported 

In [11]:
pred

Prediction(
    correct_answers=['1,234'],
    explanation='The population of Sandusky Township, Crawford County, Ohio, as of the 2020 census, was 1,234. This information is provided in Document 2 and Document 3, with Document 3 confirming the population figure from the 2010 census as well.'
)

In [12]:
import unicodedata
import re
import string
from dspy.primitives import Module

# def normalize_text(s):
#     s = unicodedata.normalize("NFD", s)

#     def remove_articles(text):
#         return re.sub(r"\b(a|an|the)\b", " ", text)

#     def white_space_fix(text):
#         return " ".join(text.split())

#     def remove_punc(text):
#         exclude = set(string.punctuation)
#         return "".join(ch for ch in text if ch not in exclude)

#     def lower(text):
#         return text.lower()

#     return white_space_fix(remove_articles(remove_punc(lower(s))))

# def evaluation_metric(example, pred):
#     llm_answers = [normalize_text(answer) for answer in pred.correct_answers]
#     gold_answers = [normalize_text(answer) for answer in example.gold_answers]
#     wrong_answers = [normalize_text(answer) for answer in example.wrong_answers]
    
#     # Check if llm_answers contains all gold answers and no wrong answers
#     all_gold_included = all(gold in llm_answers for gold in gold_answers)
#     no_wrong_included = all(wrong not in llm_answers for wrong in wrong_answers)
    
#     if all_gold_included and no_wrong_included:
#         score = 1
#     else:
#         score = 0
#     return score

class EvaluationMetric(Module):
    def __init__(self):
        pass

    def normalize_text(self, s):
        s = unicodedata.normalize("NFD", s)

        def remove_articles(text):
            return re.sub(r"\b(a|an|the)\b", " ", text)

        def white_space_fix(text):
            return " ".join(text.split())

        def remove_punc(text):
            exclude = set(string.punctuation)
            return "".join(ch for ch in text if ch not in exclude)

        def lower(text):
            return text.lower()

        return white_space_fix(remove_articles(remove_punc(lower(s))))

    def forward(self, example, pred, trace=None):
        llm_answers = [self.normalize_text(answer) for answer in pred.correct_answers]
        gold_answers = [self.normalize_text(answer) for answer in example.gold_answers]
        wrong_answers = [self.normalize_text(answer) for answer in example.wrong_answers]
        
        # Check if llm_answers contains all gold answers and no wrong answers
        all_gold_included = all(gold in llm_answers for gold in gold_answers)
        no_wrong_included = all(wrong not in llm_answers for wrong in wrong_answers)
        
        if all_gold_included and no_wrong_included:
            score = 1
        else:
            score = 0
        return score


evaluation_metric = EvaluationMetric()

evaluation_metric(example, pred)

0

In [13]:
evaluate = dspy.Evaluate(devset=devset, metric=evaluation_metric, num_threads=24,
                         display_progress=True, display_table=2)

evaluate(answer)

Average Metric: 8.00 / 50 (16.0%): 100%|██████████| 50/50 [00:00<00:00, 1091.83it/s]

2025/06/29 23:09:06 INFO dspy.evaluate.evaluate: Average Metric: 8 / 50 (16.0%)


,question,context,documents,disambig_entity,gold_answers,wrong_answers,correct_answers,explanation,EvaluationMetric
0,What is the gender composition of Wirral Grammar School?,Document 1: Wirral Grammar School for Boys Wirral Grammar School f...,"[{'text': ""Wirral Grammar School for Boys Wirral Grammar School fo...","[Wirral Grammar School for Boys, Wirral Grammar School for Girls]","[Boys, All-girls]","[Girls, Girls]","[Boys, Girls]",The gender composition of Wirral Grammar School includes both boys...,
1,When was Harry Harvey born?,"Document 1: Harry Harvey (Medal of Honor, 1865) Harry Harvey (Dece...","[{'text': 'Harry Harvey (Medal of Honor, 1865) Harry Harvey (Decem...","[Harry Harvey (Medal of Honor, 1865), Harry Harvey (Medal of Honor...","[December 14, 1846, June 4, 1873, January 10, 1901]",[],"[December 14, 1846]","The correct answer is December 14, 1846, as stated in Document 1, ...",


16.0

### Optimizer

In [14]:
tp = dspy.MIPROv2(metric=evaluation_metric, auto="medium", num_threads=24)  # use fewer threads if your rate limit is small

optimized_answer = tp.compile(answer, trainset=trainset,
                           max_bootstrapped_demos=2, max_labeled_demos=2,
                           requires_permission_to_run=False)

2025/06/29 23:09:07 INFO dspy.teleprompt.mipro_optimizer_v2: 
RUNNING WITH THE FOLLOWING MEDIUM AUTO RUN SETTINGS:
num_trials: 18
minibatch: True
num_fewshot_candidates: 12
num_instruct_candidates: 6
valset size: 160

2025/06/29 23:09:07 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/06/29 23:09:07 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used as few-shot example candidates for our program and for creating instructions.

2025/06/29 23:09:07 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=12 sets of demonstrations...


Bootstrapping set 1/12
Bootstrapping set 2/12
Bootstrapping set 3/12


  0%|          | 0/40 [00:00<?, ?it/s]/home/darth/Documents/code/ramdocs/RAMDocs/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
 32%|███▎      | 13/40 [00:00<00:00, 29.31it/s]


Bootstrapped 2 full traces after 13 examples for up to 1 rounds, amounting to 13 attempts.
Bootstrapping set 4/12


  5%|▌         | 2/40 [00:00<00:00, 579.68it/s]


...

2025/06/29 23:13:46 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 14.29 on minibatch of size 35 with parameters ['Predictor 0: Instruction 0', 'Predictor 0: Few-Shot Set 7'].
2025/06/29 23:13:46 INFO dspy.teleprompt.mipro_optimizer_v2: Minibatch scores so far: [14.29, 20.0, 28.57, 20.0, 11.43, 17.14, 20.0, 8.57, 17.14, 20.0, 34.29, 22.86, 20.0, 17.14, 11.43, 5.71, 14.29]
2025/06/29 23:13:46 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [16.88, 16.88, 17.5, 16.25]
2025/06/29 23:13:46 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 17.5
2025/06/29 23:13:46 INFO dspy.teleprompt.mipro_optimizer_v2: ==========================================


2025/06/29 23:13:46 INFO dspy.teleprompt.mipro_optimizer_v2: == Trial 22 / 23 - Minibatch ==


Average Metric: 5.00 / 35 (14.3%): 100%|██████████| 35/35 [00:00<00:00, 1683.90it/s]

2025/06/29 23:13:46 INFO dspy.evaluate.evaluate: Average Metric: 5 / 35 (14.3%)
2025/06/29 23:13:46 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 14.29 on minibatch of size 35 with parameters ['Predictor 0: Instruction 0', 'Predictor 0: Few-Shot Set 11'].
2025/06/29 23:13:46 INFO dspy.teleprompt.mipro_optimizer_v2: Minibatch scores so far: [14.29, 20.0, 28.57, 20.0, 11.43, 17.14, 20.0, 8.57, 17.14, 20.0, 34.29, 22.86, 20.0, 17.14, 11.43, 5.71, 14.29, 14.29]
2025/06/29 23:13:46 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [16.88, 16.88, 17.5, 16.25]
2025/06/29 23:13:46 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 17.5
2025/06/29 23:13:46 INFO dspy.teleprompt.mipro_optimizer_v2: ==========================================


2025/06/29 23:13:46 INFO dspy.teleprompt.mipro_optimizer_v2: ===== Trial 23 / 23 - Full Evaluation =====
2025/06/29 23:13:46 INFO dspy.teleprompt.mipro_optimizer_v2: Doing full eval on next top averaging program (Avg Score: 20


Average Metric: 24.00 / 160 (15.0%): 100%|██████████| 160/160 [00:32<00:00,  4.98it/s]

2025/06/29 23:14:18 INFO dspy.evaluate.evaluate: Average Metric: 24 / 160 (15.0%)
2025/06/29 23:14:18 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [16.88, 16.88, 17.5, 16.25, 15.0]
2025/06/29 23:14:18 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 17.5
2025/06/29 23:14:18 INFO dspy.teleprompt.mipro_optimizer_v2: =======================
2025/06/29 23:14:18 INFO dspy.teleprompt.mipro_optimizer_v2: 

2025/06/29 23:14:18 INFO dspy.teleprompt.mipro_optimizer_v2: Returning best identified program with score 17.5!


In [15]:
evaluate(optimized_answer)

Average Metric: 11.00 / 50 (22.0%): 100%|██████████| 50/50 [00:14<00:00,  3.35it/s]

2025/06/29 23:14:33 INFO dspy.evaluate.evaluate: Average Metric: 11 / 50 (22.0%)


,question,context,documents,disambig_entity,gold_answers,wrong_answers,correct_answers,explanation,EvaluationMetric
0,What is the gender composition of Wirral Grammar School?,Document 1: Wirral Grammar School for Boys Wirral Grammar School f...,"[{'text': ""Wirral Grammar School for Boys Wirral Grammar School fo...","[Wirral Grammar School for Boys, Wirral Grammar School for Girls]","[Boys, All-girls]","[Girls, Girls]","[boys, girls]",Wirral Grammar School consists of two separate institutions: Wirra...,
1,When was Harry Harvey born?,"Document 1: Harry Harvey (Medal of Honor, 1865) Harry Harvey (Dece...","[{'text': 'Harry Harvey (Medal of Honor, 1865) Harry Harvey (Decem...","[Harry Harvey (Medal of Honor, 1865), Harry Harvey (Medal of Honor...","[December 14, 1846, June 4, 1873, January 10, 1901]",[],"[December 14, 1846, June 4, 1873, January 10, 1901]",There are three individuals named Harry Harvey mentioned in the co...,✔️ [1]


22.0

In [16]:
optimized_answer.save("/home/darth/Documents/code/ramdocs/RAMDocs/src/dspy/saved_states/miro_v2_predict.json")

In [18]:
lm.inspect_history(n=1)





[2025-06-29T23:14:33.543939]

System message:

Your input fields are:
1. `question` (str): 
2. `context` (str):
Your output fields are:
1. `correct_answers` (list[str]): 
2. `explanation` (str):
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## question ## ]]
{question}

[[ ## context ## ]]
{context}

[[ ## correct_answers ## ]]
{correct_answers}        # note: the value you produce must adhere to the JSON schema: {"type": "array", "items": {"type": "string"}}

[[ ## explanation ## ]]
{explanation}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        Answer the question using the information provided in the documents. Ensure your answer is accurate and supported by the context.


User message:

[[ ## question ## ]]
What sport is Tim Martin associated with?

[[ ## context ## ]]
Document 1: Tim Martin (American football) Tim Martin (born April 12, 1976) is a former American football lineman who played 